# <B> # SageMaker pipeline </B>
* Container: codna_python3

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 0. Install packages

In [2]:
install_needed = False  # should only be True once
# install_needed = False

In [3]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

Already revised


In [4]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U smdebug sagemaker-experiments
    !{sys.executable} -m pip install -U sagemaker

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.5/718.5 kB 27.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.146.0-py2.py3-none-any.whl size=964936 sha256=d3a9669420018e3c5e20828dddd14289133eb66b2cdc3c6309016c0923d44bc4
  Stored in directory: /home/ec2-user/.cache/pip/wheels/3a/04/13/2066fc4ef9ed243c9e8710b9c269f29e7711bca655da2eb416
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.145.0
    Uninstalling sagemaker-2.145.0:
      Successfully uninstalled sagemaker-2.145.0
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


## 1. parameter store 설정

In [3]:
import boto3
from utils.ssm import parameter_store

In [4]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

In [5]:
strBucketName = pm.get_params(key="-".join([strPrefix, "BUCKET"]))
strExecutionRole = pm.get_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]))

## 2. Dataset

In [6]:
import os

In [7]:
strS3DataPath = f"s3://{strBucketName}/dataset" 
strLocalDataPath = os.path.join(os.getcwd(), "data")

## 3. MLOps pipeline
* pipeline:
    * https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_building_pipeline.html#id2 
    * [Amazon SageMaker 모델 구축 파이프라인](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines.html)
    

In [11]:
import os
import time
import sagemaker
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.processing import ProcessingInput, ProcessingOutput, FrameworkProcessor
from sagemaker.workflow.steps import CacheConfig, ProcessingStep, TrainingStep
from sagemaker.workflow.pipeline_context import PipelineSession, LocalPipelineSession
from sagemaker.workflow.retry import StepRetryPolicy, StepExceptionTypeEnum, SageMakerJobExceptionTypeEnum, SageMakerJobStepRetryPolicy

In [14]:
class pipeline():
    
    def __init__(self, bLocalMode, strPipelineName): 
        
        
        self.strRegionName = boto3.Session().region_name
        self.pm = parameter_store(self.strRegionName)
        self.strPrefix = self.pm.get_params(key="PREFIX")
        
        self.strExecutionRole = self.pm.get_params(key="-".join([self.strPrefix, "SAGEMAKER-ROLE-ARN"]))
        print (self.strExecutionRole)
        self.strBucketName = self.pm.get_params(key="-".join([self.strPrefix, "BUCKET"]))
        self.strPipelineName = strPipelineName
        self.bLocalMode = bLocalMode
     
        if self.bLocalMode:
            self.pipeline_session = LocalPipelineSession()
            self.pipeline_session.config = {'local': {'local_code': True}}
            self.strDataPath = os.path.join(os.getcwd(), "data")
        else:
            self.pipeline_session = PipelineSession()
            self.strDataPath = f"s3://{self.strBucketName}/dataset" 
        
        self.cache_config = CacheConfig(
            enable_caching=True,
            expire_after="T48H"
        )
        
        self.retry_policies=[                
            # retry when resource limit quota gets exceeded
            SageMakerJobStepRetryPolicy(
                exception_types=[SageMakerJobExceptionTypeEnum.RESOURCE_LIMIT],
                expire_after_mins=180,
                interval_seconds=60,
                backoff_rate=1.0
            ),
        ]
        
    def _step_training(self, ):
        
        
        strInstanceType = "ml.m5.xlarge"
        nInstanceCount = 1
        bSpotTraining = False
        
        if self.bLocalMode:
            dicDataChannels = {
                "training": f'file://{os.path.join(self.strDataPath, "train.csv")}',
                "testing": f'file://{os.path.join(self.strDataPath, "test.csv")}',
            }
        else:
            dicDataChannels = {
                "training": os.path.join(self.strDataPath, "train.csv"),
                "testing": os.path.join(self.strDataPath, "test.csv"),
            }
            
        if bSpotTraining:
            nMaxWait = 1*60*60
            nMaxRun = 1*60*60
        else:
            nMaxWait = None
            nMaxRun = 1*60*60
    
        strOutputPath = os.path.join(
            "s3://{}".format(self.strBucketName),
            self.strPipelineName,
            "training",
            "model-output"
        )

        strCodeLocation = os.path.join(
            "s3://{}".format(self.strBucketName),
            self.strPipelineName,
            "training",
            "backup_codes"
        )
        
        dicHyperparameters = {
            "scale_pos_weight" : "19",    
            "max_depth": "2",
            "eta": "0.3",
            "objective": "binary:logistic",
            "num_round": "100",
        }
        
        self.estimator = XGBoost(
            entry_point="xgboost_starter_script.py",
            source_dir="source/train/",
            output_path=strOutputPath,
            code_location=strCodeLocation,
            hyperparameters=dicHyperparameters, ## Contatiner내 env. variable로 들어 감
            role=self.strExecutionRole,
            instance_count=nInstanceCount,
            instance_type=strInstanceType,
            framework_version="1.3-1",
            max_run=nMaxRun,
            use_spot_instances=bSpotTraining,
            max_wait=nMaxWait,
            #keep_alive_period_in_seconds=nKeepAliveSeconds,
            enable_sagemaker_metrics=True,
            volume_size=64, ## GB
            
            sagemaker_session=self.pipeline_session
            
        )
        
        job_name = "-".join([self.strPipelineName, "training-job"])
        step_training_args = self.estimator.fit(
            inputs=dicDataChannels,
            job_name=job_name,
            experiment_config={
              'TrialName': job_name,
              'TrialComponentDisplayName': job_name,
            },
            logs="All",
        )
        
        self.training_process = TrainingStep(
            name="TrainingProcess",
            step_args=step_training_args,
            cache_config=self.cache_config,
            retry_policies=self.retry_policies
        )
            
        print ("  \n== Training Step ==")
        print ("   \nArgs: ", self.training_process.arguments.items())   
        
    
    def _step_evaluation(self, ):
        
        if self.bLocalMode: pipeline_session = LocalPipelineSession() ## processing job에서 local code는 적용되지 않기 때문
        else: pipeline_session=self.pipeline_session 
        
        strInstanceType = "ml.m5.xlarge"
        nInstanceCount = 1
        
        strProcPrefixPath = "/opt/ml/processing"
        strTestDataPath = os.path.join(self.strDataPath, "test.csv")
        
        strOutputPath = os.path.join(
            "s3://{}".format(self.strBucketName),
            self.strPipelineName,
            "evaluation",
            "output"
        )

        strCodeLocation = os.path.join(
            "s3://{}".format(self.strBucketName),
            self.strPipelineName,
            "evaluation",
            "backup_codes"
        )
        
        evaluation_processor = FrameworkProcessor(
            estimator_cls=XGBoost,
            framework_version="1.3-1",
            image_uri=None,
            role=self.strExecutionRole,
            instance_type=strInstanceType,
            instance_count=nInstanceCount,
            base_job_name="evaluation", # bucket에 보이는 이름 (pipeline으로 묶으면 pipeline에서 정의한 이름으로 bucket에 보임)
            
            sagemaker_session=pipeline_session
        )
        
        step_evaluation_args = evaluation_processor.run(
            code="evaluation.py",
            source_dir="source/evaluation/",
            inputs=[
                ProcessingInput(
                    source=strTestDataPath,
                    input_name="test_data",
                    destination=os.path.join(strProcPrefixPath, "test")
                ),
                ProcessingInput(
                    source=self.training_process.properties.ModelArtifacts.S3ModelArtifacts,
                    input_name="model_weight",
                    destination=os.path.join(strProcPrefixPath, "model")
                )
            ],
            outputs=[
                ProcessingOutput(
                    source=os.path.join(strProcPrefixPath, "output"),
                    output_name='evaluation',
                    destination=strOutputPath,
                )
            ],
        )
        
        self.evaluation_report = PropertyFile(
            name="EvaluationReport",
            output_name="evaluation", ## evaluation의 ProcessingOutput의 output_name
            path="evaluation.json", ## evaluate.py 에서 write하는 부분
        )
        
        self.evaluation_process = ProcessingStep(
            name="EvaluationProcess", ## Processing job이름
            step_args=step_evaluation_args,
            depends_on=[self.training_process],
            property_files=[self.evaluation_report],
            cache_config=self.cache_config,
            retry_policies=self.retry_policies
        )
    
        print ("  \n== Evaluation Step ==")
        print ("   \nArgs: ", self.evaluation_process.arguments.items())
        
    
    def _step_deploy(self, ):
        
        if self.bLocalMode: pipeline_session = LocalPipelineSession() ## processing job에서 local code는 적용되지 않기 때문
        else: pipeline_session=self.pipeline_session 
        
        strInstanceType = "ml.m5.xlarge"
        nInstanceCount = 1
        strEndpointName = f"endpoint--{self.strPipelineName}{int(time.time())}"
        strProcPrefixPath = "/opt/ml/processing"
        
        deploy_processor = FrameworkProcessor(
            estimator_cls=XGBoost,
            framework_version="1.3-1",
            image_uri=None,
            role=self.strExecutionRole,
            instance_type=strInstanceType,
            instance_count=nInstanceCount,
            base_job_name="deploy", # bucket에 보이는 이름 (pipeline으로 묶으면 pipeline에서 정의한 이름으로 bucket에 보임)
            sagemaker_session=pipeline_session
        )
        
        step_deploy_args = deploy_processor.run(
            code="deploy.py",
            source_dir="source/deploy/",
            inputs=[
                ProcessingInput(
                    source="source/deploy/inference.py",
                    input_name="inference-py",
                    destination=os.path.join(strProcPrefixPath, "inference")
                ),
            ],
            arguments=[
                "--prefix_deploy", strProcPrefixPath, \
                "--region", self.strRegionName, \
                "--instance_type", strInstanceType, \
                "--model_data", self.training_process.properties.ModelArtifacts.S3ModelArtifacts, \
                "--endpoint_name", strEndpointName, \
                "--execution_role", self.strExecutionRole, \
                "--local_mode", str(self.bLocalMode), \
            ],
            job_name="deploy",
        )
        
        self.pm.put_params(key=self.strPrefix + "ENDPOINT-NAME", value=strEndpointName, overwrite=True)
        
        self.deploy_process = ProcessingStep(
            name="DeployProcess", ## Processing job이름
            step_args=step_deploy_args,
            depends_on=[self.evaluation_process],
            cache_config=self.cache_config,
            retry_policies=self.retry_policies
        )
        
        print ("  \n== Deploy Step ==")
        print ("   \nArgs: ", self.deploy_process.arguments.items())
    
    def _get_pipeline(self, ):
        
        if self.bLocalMode: steps=[self.training_process, self.evaluation_process]
        else: steps=[self.training_process, self.evaluation_process, self.deploy_process]
            
        pipeline = Pipeline(
            name=self.strPipelineName,
            steps=steps,
            sagemaker_session=self.pipeline_session
        )

        return pipeline
                      
    def execution(self, ):
    
        self._step_training()
        self._step_evaluation()
        if not self.bLocalMode: self._step_deploy()
        
        pipeline = self._get_pipeline()
        pipeline.upsert(role_arn=self.strExecutionRole) ## Submit the pipeline definition to the SageMaker Pipelines service 
        execution = pipeline.start()
        execution.describe()
     

In [15]:
pipe = pipeline(
    bLocalMode=True,
    strPipelineName=f'{strPrefix}-PIPELINE'
)
pipe.execution()

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.m5.xlarge.
INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.m5.xlarge.
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436
  
== Training Step ==
   
Args:  dict_items([('AlgorithmSpecification', {'TrainingInputMode': 'File', 'TrainingImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1', 'EnableSageMakerMetricsTimeSeries': True}), ('OutputDataConfig', {'S3OutputPath': 's3://sagemaker-us-east-1-419974056037/DJ-SM-IMD-PIPELINE/training/model-output'}), ('StoppingCondition', {'MaxRuntimeInSeconds': 3600}), ('ResourceConfig', {'VolumeSizeInGB': 64, 'InstanceCount': 1, 'InstanceType': 'ml.m5.xlarge'}), ('RoleArn', 'arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436'), ('InputDataConfig', [{'DataSource': {'FileDataSource': {'FileDataDistributionType': 'FullyReplicated', 'FileUri': 'file:///home/ec2-user/SageMaker/sagemaker-immersion-day/data/train.csv'}}, 'ChannelName': 'training'}, {'DataSource': {'FileDataSource': {'FileDataDistributionType': 'FullyReplicate

INFO:sagemaker.processing:Uploaded source/evaluation/ to s3://sagemaker-us-east-1-419974056037/evaluation-2023-04-16-07-15-58-843/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-419974056037/evaluation-2023-04-16-07-15-58-843/source/runproc.sh
INFO:sagemaker.processing:Uploaded source/evaluation/ to s3://sagemaker-us-east-1-419974056037/evaluation-2023-04-16-07-15-59-530/source/sourcedir.tar.gz


   
Args:  dict_items([('ProcessingResources', {'ClusterConfig': {'InstanceType': 'ml.m5.xlarge', 'InstanceCount': 1, 'VolumeSizeInGB': 30}}), ('AppSpecification', {'ImageUri': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1', 'ContainerEntrypoint': ['/bin/bash', '/opt/ml/processing/input/entrypoint/runproc.sh']}), ('RoleArn', 'arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436'), ('ProcessingInputs', [{'InputName': 'test_data', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-419974056037/evaluation-2023-04-16-07-15-58-843/input/test_data/test.csv', 'LocalPath': '/opt/ml/processing/test', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'model_weight', 'AppManaged': False, 'S3Input': {'S3Uri': <sagemaker.workflow.properties.Properties object at 0x7f7bc1056230>, 'LocalPath': '/opt/ml/processing/model', 'S3DataType': 'S

INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-419974056037/evaluation-2023-04-16-07-15-59-530/source/runproc.sh
INFO:sagemaker.processing:Uploaded source/evaluation/ to s3://sagemaker-us-east-1-419974056037/evaluation-2023-04-16-07-15-59-779/source/sourcedir.tar.gz


...

s215drh6jr-algo-1-4k4pr exited with code 0
Aborting on container exit...


INFO:sagemaker.processing:Uploaded source/evaluation/ to s3://sagemaker-us-east-1-419974056037/evaluation-2023-04-16-07-16-11-398/source/sourcedir.tar.gz


===== Job Complete =====
Pipeline step 'TrainingProcess' SUCCEEDED.
Starting pipeline step: 'EvaluationProcess'


INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-419974056037/evaluation-2023-04-16-07-16-11-398/source/runproc.sh
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:270: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(
INFO:sagemaker.local.local_session:Starting processing job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-ele1q:
    container_name: ue08vgn5x8-algo-1-ele1q
    entrypoint:
    - /bin/bash
    - /opt/ml/processing/input/entrypoint/runproc.sh
    environment: []
    image: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:

Creating ue08vgn5x8-algo-1-ele1q ... 
Creating ue08vgn5x8-algo-1-ele1q ... done
Attaching to ue08vgn5x8-algo-1-ele1q
ue08vgn5x8-algo-1-ele1q | Requirement already satisfied: xgboost in /miniconda3/lib/python3.7/site-packages (1.3.3)
ue08vgn5x8-algo-1-ele1q | Requirement already satisfied: numpy in /miniconda3/lib/python3.7/site-packages (from xgboost) (1.21.6)
ue08vgn5x8-algo-1-ele1q | Requirement already satisfied: scipy in /miniconda3/lib/python3.7/site-packages (from xgboost) (1.6.2)
ue08vgn5x8-algo-1-ele1q | WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv
ue08vgn5x8-algo-1-ele1q | 
ue08vgn5x8-algo-1-ele1q | [notice] A new release of pip is available: 23.0.1 -> 23.1
ue08vgn5x8-algo-1-ele1q | [notice] To update, run: pip install --upgrade pip
ue08vgn5x8-algo-1-ele1q | #############################################